<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/1.%20RestructureFaveOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set Up**

In [1]:
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


**1. Load Data Files**

In [4]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m', 'C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']
condition = ['reading', 'interview']
extention = ['txt', 'TextGrid', 'csv']

In [5]:
# get required shape of dataframes
max_read_txt = 0
max_read_TextGrid = 0
max_read_csv = 0 
max_inter_txt = 0
max_inter_TextGrid = 0
max_inter_csv = 0

In [6]:
for ID in IDs:
  for cond in condition:
    for ext in extention: 
      if cond == 'reading':
        if ext == 'txt': 
          if len(pd.read_csv(ID + '_' + cond + '.' + ext)) > max_read_txt: 
            max_read_txt = len(pd.read_csv(ID + '_' + cond + '.' + ext))
        elif ext == 'TextGrid':
          if len(pd.read_csv(ID + '_' + cond + '.' + ext)) > max_read_TextGrid: 
            max_read_TextGrid = len(pd.read_csv(ID + '_' + cond + '.' + ext))
        elif ext == "csv":
          if len(pd.read_csv(ID + '_FS_' + cond + '.' + ext)) > max_read_csv: 
            max_read_csv = len(pd.read_csv(ID + '_FS_' + cond + '.' + ext))
      elif cond == 'interview':
        if ext == 'txt': 
          if len(pd.read_csv(ID + '_' + cond + '.' + ext)) > max_inter_txt: 
            max_inter_txt = len(pd.read_csv(ID + '_' + cond + '.' + ext))
        elif ext == 'TextGrid':
          if len(pd.read_csv(ID + '_' + cond + '.' + ext)) > max_inter_TextGrid: 
            max_inter_TextGrid = len(pd.read_csv(ID + '_' + cond + '.' + ext))
        elif ext == 'csv':
          if len(pd.read_csv(ID + '_FS_' + cond + '.' + ext)) <  max_read_csv:
            max_inter_csv = max_inter_csv
          elif len(pd.read_csv(ID + '_FS_' + cond + '.' + ext)) > max_inter_csv: 
            max_inter_csv = len(pd.read_csv(ID + '_FS_' + cond + '.' + ext))

In [7]:
# initialize dataframes
df_read_txt = pd.DataFrame(index=range(max_read_txt),columns=range(1))
df_read_TextGrid = pd.DataFrame(index=range(max_read_TextGrid),columns=range(1))
df_read_csv = pd.DataFrame(index=range(max_read_csv), columns=range(1))
df_inter_txt = pd.DataFrame(index=range(max_inter_txt),columns=range(1))
df_inter_TextGrid = pd.DataFrame(index=range(max_inter_TextGrid),columns=range(1))
df_inter_csv = pd.DataFrame(index=range(max_inter_csv), columns=range(1))

In [8]:
#import data
for ID in IDs:
  for cond in condition:
    for ext in extention: 
      if cond == 'reading':
        if ext == 'txt': 
          df_read_txt[ID] = pd.read_csv(ID + '_' + cond + '.' + ext, header = None)
        elif ext == 'TextGrid':
          df_read_TextGrid[ID] = pd.read_csv(ID + '_' + cond + '.' + ext)
        elif ext == 'csv':
          df_read_csv[ID] = pd.read_csv(ID + '_FS_' + cond + '.' + ext)
      elif cond == 'interview':
        if ext == 'txt': 
          df_inter_txt[ID] = pd.read_csv(ID + '_' + cond + '.' + ext, header = None)
        elif ext == 'TextGrid':
          df_inter_TextGrid[ID] = pd.read_csv(ID + '_' + cond + '.' + ext)
        elif ext == 'csv':
          df_inter_csv[ID] = pd.read_csv(ID + '_FS_' + cond + '.' + ext)

In [9]:
df_read_txt_wide = pd.DataFrame()
df_inter_txt_wide = pd.DataFrame()
df_read_TextGrid_wide = pd.DataFrame()
df_inter_TextGrid_wide = pd.DataFrame()

**2. Combine txt Files**

In [10]:
for ID in IDs:
  df_read_txt_wide = df_read_txt_wide.append(df_read_txt[ID].str.split("\t", expand = True, n = 4), ignore_index=True) 
  df_inter_txt_wide = df_inter_txt_wide.append(df_inter_txt[ID].str.split("\t", expand = True, n = 4), ignore_index=True) 

In [11]:
df_read_txt_wide.drop(columns=[0, 1], inplace = True)
df_read_txt_wide.rename(columns={2: 'Onset', 3: 'Offset', 4: "Utterance"}, inplace=True)
df_inter_txt_wide.drop(columns=[0, 1], inplace = True)
df_inter_txt_wide.rename(columns={2: 'Onset', 3: 'Offset', 4: "Utterance"}, inplace=True)

In [12]:
names_read = []
name_col_read = []
names_inter = []
name_col_inter = []

In [13]:
for ID in IDs:
  names_read.append([ID for i in range(len(df_read_txt))])
for n in range(0,len(IDs)):
  name_col_read = name_col_read + names_read[n]

In [14]:
for ID in IDs:
  names_inter.append([ID for i in range(len(df_inter_txt))])
for n in range(0,len(IDs)):
  name_col_inter = name_col_inter + names_inter[n]

In [15]:
df_read_txt_wide.insert(0, 'ID', name_col_read)
df_inter_txt_wide.insert(0, 'ID', name_col_inter)

In [16]:
df_read_txt_wide.dropna(inplace=True)
df_inter_txt_wide.dropna(inplace=True)

In [17]:
# add Fluencystatus column in reading
FS_col = pd.DataFrame()
for ID in IDs:
  subset_ID = df_read_txt_wide[df_read_txt_wide["ID"] == ID]
  subset_ID.index = range(len(subset_ID.index)) # reset index
  FS = df_read_csv[ID].head(len(subset_ID)) # get FS without NANs
  subset_ID["FluencyStatus"] = FS               # add to sub dataframe
  FS_col = FS_col.append([subset_ID], ignore_index = True)
df_read_txt_wide = FS_col

In [18]:
df_read_txt_wide['FluencyStatus'].isnull().sum()  # should be zero

0

In [19]:
# add Fluencystatus column in interview
FS_col = pd.DataFrame()
for ID in IDs:
  subset_ID = df_inter_txt_wide[df_inter_txt_wide["ID"] == ID]
  subset_ID.index = range(len(subset_ID.index)) # reset index
  FS = df_inter_csv[ID].head(len(subset_ID)) # get FS without NANs
  subset_ID["FluencyStatus"] = FS               # add to sub dataframe
  FS_col = FS_col.append([subset_ID], ignore_index = True)
df_inter_txt_wide = FS_col

In [20]:
df_inter_txt_wide['FluencyStatus'].isnull().sum()  # should be zero

0

**3.1 Combine TextGrid Files: Reading**

In [21]:
IntervalSizeString = []

In [22]:
for ID in IDs: 
  IntervalSizeString.append(df_read_TextGrid[ID][11])

In [23]:
IntervalSize = []
for i in range(0,len(IntervalSizeString)):
  for z in IntervalSizeString[i].split():
    if z.isdigit():
        IntervalSize.append(int(z))

In [24]:
maxLength = max(IntervalSize)
lastRow = len(df_read_TextGrid)
n = -1

In [25]:
for ID in IDs:  
  n = n + 1
  endRow = IntervalSize[n] * 4 + 12
  df_read_TextGrid[ID][endRow:lastRow] = np.NaN

In [26]:
for c in range(0,12):
  df_read_TextGrid.drop([c], inplace = True)

In [27]:
for ID in IDs:
  df_read_TextGrid_wide = df_read_TextGrid_wide.append(df_read_TextGrid[ID].str.split(" = ", expand = True, n = 1), ignore_index=True) 

In [28]:
df_read_TextGrid_wide = df_read_TextGrid_wide[df_read_TextGrid_wide[0].str.contains("xmin|xmax|text") == True]

In [29]:
onsets = df_read_TextGrid_wide[df_read_TextGrid_wide[0].str.contains("xmin") == True]
offsets = df_read_TextGrid_wide[df_read_TextGrid_wide[0].str.contains("xmax") == True]
sounds = df_read_TextGrid_wide[df_read_TextGrid_wide[0].str.contains("text") == True]

In [30]:
onsets.index = range(len(onsets.index))
offsets.index = range(len(offsets.index))
sounds.index = range(len(sounds.index))

In [31]:
duration = []

In [32]:
for i in range (0,(len(offsets))):
  duration.append(float(offsets[1][i]) - float(onsets[1][i]))

In [33]:
duration = pd.DataFrame(duration)

In [34]:
read_TextGrid_duration = pd.concat([sounds, onsets, offsets, duration], axis=1)

In [35]:
read_TextGrid_duration.columns = ['text', 'Sound', 'min', 'Onset', 'max','Offset','Duration']

In [36]:
read_TextGrid_duration.drop(['text', 'min', 'max'], axis=1, inplace = True)

In [37]:
ID_col = pd.DataFrame(columns=range(1))

In [38]:
n = -1

In [39]:
for ID in IDs:
  n = n + 1
  ID_col = ID_col.append([ID]*IntervalSize[n])

In [40]:
ID_col.index = range(len(ID_col.index))

In [41]:
read_TextGrid_duration = pd.concat([ID_col, read_TextGrid_duration], axis=1)

In [42]:
read_TextGrid_duration.columns = ['ID','Sound', 'Onset', 'Offset','Duration']

In [43]:
read_TextGrid_duration

,ID,Sound,Onset,Offset,Duration
0,24fa,"""sp""",0.000000,0.013000,0.013
1,24fa,"""sp""",0.013000,0.143000,0.130
2,24fa,"""S""",0.143000,0.263000,0.120
3,24fa,"""OW1""",0.263000,0.343000,0.080
4,24fa,"""HH""",0.343000,0.482000,0.139
...,...,...,...,...,...
24338,C13_RG,"""N""",87.881000,87.910000,0.029
24339,C13_RG,"""EY2""",87.910000,88.081000,0.171
24340,C13_RG,"""M""",88.081000,88.191000,0.110
24341,C13_RG,"""Z""",88.191000,88.281000,0.090


**3.2 Combine TextGrid Files: Interview**

In [44]:
IntervalSizeString = []

In [45]:
for ID in IDs: 
  IntervalSizeString.append(df_inter_TextGrid[ID][11])

In [46]:
IntervalSize = []
for i in range(0,len(IntervalSizeString)):
  for z in IntervalSizeString[i].split():
    if z.isdigit():
        IntervalSize.append(int(z))

In [47]:
maxLength = max(IntervalSize)
lastRow = len(df_inter_TextGrid)
n = -1

In [48]:
for ID in IDs:  
  n = n + 1
  endRow = IntervalSize[n] * 4 + 12
  df_inter_TextGrid[ID][endRow:lastRow] = np.NaN

In [49]:
for c in range(0,12):
  df_inter_TextGrid.drop([c], inplace = True)

In [50]:
for ID in IDs:
  df_inter_TextGrid_wide = df_inter_TextGrid_wide.append(df_inter_TextGrid[ID].str.split(" = ", expand = True, n = 1), ignore_index=True) 

In [51]:
df_inter_TextGrid_wide = df_inter_TextGrid_wide[df_inter_TextGrid_wide[0].str.contains("xmin|xmax|text") == True]

In [52]:
onsets = df_inter_TextGrid_wide[df_inter_TextGrid_wide[0].str.contains("xmin") == True]
offsets = df_inter_TextGrid_wide[df_inter_TextGrid_wide[0].str.contains("xmax") == True]
sounds = df_inter_TextGrid_wide[df_inter_TextGrid_wide[0].str.contains("text") == True]

In [53]:
onsets.index = range(len(onsets.index))
offsets.index = range(len(offsets.index))
sounds.index = range(len(sounds.index))

In [54]:
duration = []

In [55]:
for i in range (0,(len(offsets))):
  duration.append(float(offsets[1][i]) - float(onsets[1][i]))

In [56]:
duration = pd.DataFrame(duration)

In [57]:
inter_TextGrid_duration = pd.concat([sounds, onsets, offsets, duration], axis=1)

In [58]:
inter_TextGrid_duration.columns = ['text', 'Sound', 'min', 'Onset', 'max','Offset','Duration']

In [59]:
inter_TextGrid_duration.drop(['text', 'min', 'max'], axis=1, inplace = True)

In [60]:
ID_col = pd.DataFrame(columns=range(1))

In [61]:
n = -1

In [62]:
for ID in IDs:
  n = n + 1
  ID_col = ID_col.append([ID]*IntervalSize[n])

In [63]:
ID_col.index = range(len(ID_col.index))

In [64]:
inter_TextGrid_duration = pd.concat([ID_col, inter_TextGrid_duration], axis=1)

In [65]:
inter_TextGrid_duration.columns = ['ID', 'Sound', 'Onset', 'Offset','Duration']

In [66]:
inter_TextGrid_duration

,ID,Sound,Onset,Offset,Duration
0,24fa,"""sp""",0.000000,0.027000,0.027
1,24fa,"""sp""",0.027000,0.127000,0.100
2,24fa,"""AY1""",0.127000,0.187000,0.060
3,24fa,"""M""",0.187000,0.277000,0.090
4,24fa,"""IY1""",0.277000,0.327000,0.050
...,...,...,...,...,...
76292,C13_RG,"""IH2""",163.144000,163.214000,0.070
76293,C13_RG,"""P""",163.214000,163.284000,0.070
76294,C13_RG,"""S""",163.284000,163.584000,0.300
76295,C13_RG,"""sp""",163.584000,163.684000,0.100


**4. Add Info to TextGrid Files**

**4.1 Sound Inventory**

(May throw error if new file contains additional sounds)

In [67]:
arpabet = np.array(['"sp"', '"AY1"', '"M"', '"IY1"', '"N"', '"IH1"', '"T"', '"S"', '"D"', '"EH1"', '"F"', '"AH0"', '"L"', '"IY0"', '"K"', '"EH0"', '"ER0"', '"AH1"', '"P"', '"AE1"', '"Y"', '"R"', '"W"', '"Z"',
       '"EY1"', '"UH1"', '"AA0"', '"EY2"', '"V"', '"CH"', '"JH"', '"OW1"', '"B"', '"IH0"', '"DH"', '"NG"', '"AA1"', '"ER2"', '"UW1"', '"HH"','"G"', '"SH"', '"ER1"', '"AW1"', '"TH"', '"AO1"', '"IH2"', '"UW0"',
       '"EH2"', '"UW2"', '"AH2"', '"OY1"', '"UH2"', '"AY2"', '"IY2"','"OW2"', '"AW2"', '"UH0"', '"OW0"', '"AE0"', '"AY0"', '"EY0"', '"AO0"', '"AA2"', '"AE2"', '"AO2"', '"ZH"', '"sp" ', '"R" ','"IY1" ', '"L" ', 
       '"IY0" ', '"B" ', '"AE1" ', '"D" ', '"S" ', '"AH1" ', '"M" ', '"UW1" ', '"AH0" ', '"AY2" ', '"T" ', '"EH1" ', '"P" ', '"AY1" ', '"F" ', '"N" ', '"Z" ', '"HH" ', '"V" ', '"AO1" ', '"W" ', '"G" ', '"EY1" ', 
       '"DH" ', '"OW1" ', '"IH1" ', '"AA1" ', '"K" ', '"ER0" ', '"AW1" ', '"IH0" ', '"JH" ', '"OY1" ', '"TH" ', '"ER1" ', '"Y" ', '"CH" ', '"UH1" ', '"OW0" ', '"NG" ', '"IH2" ', '"EH0" ', '"AE2" ', '"SH" ', 
       '"UW0" '])

In [68]:
vowels = np.array(['"AY1"', '"IY1"', '"IH1"',  '"EH1"', '"AH0"', '"IY0"', '"EH0"', '"ER0"', '"AH1"', '"AE1"', '"EY1"', '"UH1"', '"AA0"', '"EY2"', '"OW1"', '"IH0"', '"AA1"', '"ER2"', '"UW1"', '"ER1"', '"AW1"', 
                   '"AO1"', '"IH2"', '"UW0"','"EH2"', '"UW2"', '"AH2"', '"OY1"', '"UH2"', '"AY2"', '"IY2"','"OW2"', '"AW2"', '"UH0"', '"OW0"', '"AE0"', '"AY0"', '"EY0"', '"AO0"', '"AA2"', '"AE2"', '"AO2"',
                   '"IY1" ', '"L" ', '"IY0" ', '"AE1" ', '"AH1" ', '"UW1" ', '"AH0" ', '"AY2" ', '"EH1" ', '"AY1" ', '"AO1" ', '"EY1" ', '"OW1" ', '"IH1" ', '"AA1" ', '"ER0" ', '"AW1" ', '"IH0" ', '"OY1" ', 
                   '"ER1" ', '"UH1" ', '"OW0" ', '"IH2" ', '"EH0" ', '"AE2" ', '"UW0" '])

In [69]:
consonants = np.array(['"M"', '"N"', '"T"', '"S"', '"D"', '"F"', '"L"', '"K"', '"P"', '"Y"', '"R"', '"W"', '"Z"', '"V"', '"CH"', '"JH"',  '"B"', '"DH"', '"NG"', '"HH"','"G"', '"SH"', '"TH"', '"ZH"', '"R" ', 
                       '"L" ', '"B" ', '"D" ', '"S" ', '"M" ', '"T" ', '"P" ', '"F" ', '"N" ', '"Z" ', '"HH" ', '"V" ', '"W" ', '"G" ', '"DH" ', '"K" ', '"JH" ', '"TH" ', '"Y" ', '"CH" ', '"NG" ', '"SH" ', ])

In [70]:
silence = np.array(['"sp"', '"sp" '])

**4.2 Assign Sound Type to New Column**

Reading

In [71]:
Type_col = pd.DataFrame(columns=range(1))
Type_col.columns = ["Type"]

In [72]:
for i in range(0, len(read_TextGrid_duration)):
  if read_TextGrid_duration["Sound"][i] in vowels:
    Type_col = Type_col.append({"Type":"vowel"}, ignore_index=True)
  elif read_TextGrid_duration["Sound"][i] in consonants:
    Type_col = Type_col.append({"Type":"consonant"}, ignore_index=True)
  elif read_TextGrid_duration["Sound"][i] in silence:
    Type_col = Type_col.append({"Type":"silence"}, ignore_index=True)

In [73]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, Type_col], axis=1)

In [74]:
read_TextGrid_duration = read_TextGrid_duration[["ID", "Type", "Sound", "Onset", "Offset", "Duration"]]

In [75]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration
0,24fa,silence,"""sp""",0.000000,0.013000,0.013
1,24fa,silence,"""sp""",0.013000,0.143000,0.130
2,24fa,consonant,"""S""",0.143000,0.263000,0.120
3,24fa,vowel,"""OW1""",0.263000,0.343000,0.080
4,24fa,consonant,"""HH""",0.343000,0.482000,0.139
...,...,...,...,...,...,...
24338,C13_RG,consonant,"""N""",87.881000,87.910000,0.029
24339,C13_RG,vowel,"""EY2""",87.910000,88.081000,0.171
24340,C13_RG,consonant,"""M""",88.081000,88.191000,0.110
24341,C13_RG,consonant,"""Z""",88.191000,88.281000,0.090


**4.2 Assign Sound Type to New Column**

Interview

In [76]:
Type_col = pd.DataFrame(columns=range(1))
Type_col.columns = ["Type"]

In [77]:
for i in range(0, len(inter_TextGrid_duration)):
  if inter_TextGrid_duration["Sound"][i] in vowels:
    Type_col = Type_col.append({"Type":"vowel"}, ignore_index=True)
  elif inter_TextGrid_duration["Sound"][i] in consonants:
    Type_col = Type_col.append({"Type":"consonant"}, ignore_index=True)
  elif inter_TextGrid_duration["Sound"][i] in silence:
    Type_col = Type_col.append({"Type":"silence"}, ignore_index=True)

In [78]:
inter_TextGrid_duration = pd.concat([inter_TextGrid_duration, Type_col], axis=1)

In [79]:
inter_TextGrid_duration = inter_TextGrid_duration[["ID", "Type", "Sound", "Onset", "Offset", "Duration"]]

In [80]:
inter_TextGrid_duration 

,ID,Type,Sound,Onset,Offset,Duration
0,24fa,silence,"""sp""",0.000000,0.027000,0.027
1,24fa,silence,"""sp""",0.027000,0.127000,0.100
2,24fa,vowel,"""AY1""",0.127000,0.187000,0.060
3,24fa,consonant,"""M""",0.187000,0.277000,0.090
4,24fa,vowel,"""IY1""",0.277000,0.327000,0.050
...,...,...,...,...,...,...
76292,C13_RG,vowel,"""IH2""",163.144000,163.214000,0.070
76293,C13_RG,consonant,"""P""",163.214000,163.284000,0.070
76294,C13_RG,consonant,"""S""",163.284000,163.584000,0.300
76295,C13_RG,silence,"""sp""",163.584000,163.684000,0.100


**5. Save Combined Data Files**

In [81]:
dir = "1.CombinedInputFiles"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [82]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles


In [83]:
df_read_txt_wide.to_excel("read_txt_comb.xlsx")
df_inter_txt_wide.to_excel("interview_txt_comb.xlsx")
read_TextGrid_duration.to_excel("read_TextGrid_comb.xlsx")
inter_TextGrid_duration.to_excel("interview_TextGrid_comb.xlsx")

In [84]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline
